In [1]:
from libraries import GRU_model
from libraries import get_torch_data
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score
import pickle as pkl

data_path = 'data'

In [2]:
train_data = get_torch_data.get_train_data(data_path)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [3]:
from hyperopt import fmin, tpe, hp, rand, Trials, STATUS_OK
from hyperopt.pyll import scope

In [4]:
def objective(params):
    print(params)

    colsample=params['colsample']
    dense1_size=params['dense1_size']
    GRU_size=params['GRU_size']
    dense2_size=params['dense2_size']
    dense3_size=params['dense3_size']
    dense4_size=params['dense4_size']
    dense5_size=params['dense5_size']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']

    model = GRU_model.Model(seed=24338, features=train_data.features,
        colsample=colsample, dense1_size=dense1_size, GRU_size=GRU_size,
        dense2_size=dense2_size, dense3_size=dense3_size,
        dense4_size=dense4_size, dense5_size=dense5_size)
    model.to(device)
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    trial = GRU_model.TrainTrial(batch_size=batch_size, max_n_epochs=200, patience=7,
        min_validation_rate=.1, train_data=train_data, seed=35375)

    model = trial.train(device=device, model=model, optimizer=optimizer,
        objective_function=criterion, validation_metric=roc_auc_score)
    
    return {'loss': - trial.best_validation_score, 'status': STATUS_OK}

In [15]:
space = {'colsample': hp.uniform('colsample', .2, .7),
    'batch_size': scope.int(hp.quniform('batch_size', 500, 4001, 500)),
    'learning_rate': hp.loguniform('learning_rate', -4, 0),
    'dense1_size': 400,
    'GRU_size': 200,
    'dense2_size': 200,
    'dense3_size': 150,
    'dense4_size': 100,
    'dense5_size': 40}

In [16]:
#trials = Trials()
#search = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=70, trials=trials)

{'GRU_size': 200, 'batch_size': 1000, 'colsample': 0.5268300594440773, 'dense1_size': 400, 'dense2_size': 200, 'dense3_size': 150, 'dense4_size': 100, 'dense5_size': 40, 'learning_rate': 0.12375578105906028}
Early stopping.                                       
{'GRU_size': 200, 'batch_size': 2500, 'colsample': 0.431616033120658, 'dense1_size': 400, 'dense2_size': 200, 'dense3_size': 150, 'dense4_size': 100, 'dense5_size': 40, 'learning_rate': 0.03596060662907992}
Epoch 10:                                                           
	triggers: 4                                                        
	best score: 0.9343297525465031                                     
	8.77 seconds per epoch on average.                                 
Early stopping.                                                     
{'GRU_size': 200, 'batch_size': 4000, 'colsample': 0.24253123103830665, 'dense1_size': 400, 'dense2_size': 200, 'dense3_size': 150, 'dense4_size': 100, 'dense5_size': 40, 'learning_rate

In [18]:
#with open('first_search.pickle', 'wb') as file:
#    pkl.dump(trials, file)

In [16]:
with open('first_search.pickle', 'rb') as file:
    trials = pkl.load(file)

In [17]:
learning_params = {k:v[0] for k, v in trials.best_trial['misc']['vals'].items()}

In [18]:
layers_param = {'dense1_size': scope.int(hp.quniform('dense1_size', 100, 501, 100)),
    'GRU_size': scope.int(hp.quniform('GRU_size', 100, 501, 100)),
    'dense2_size': scope.int(hp.quniform('dense2_size', 100, 501, 100)),
    'dense3_size': scope.int(hp.quniform('dense3_size', 100, 501, 100)),
    'dense4_size': scope.int(hp.quniform('dense4_size', 100, 501, 100)),
    'dense5_size': scope.int(hp.quniform('dense5_size', 100, 501, 100))}

In [19]:
space = layers_param | learning_params

In [20]:
space['batch_size'] = int(space['batch_size'])

In [21]:
trials = Trials()
search = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=70, trials=trials)

{'GRU_size': 100, 'batch_size': 1500, 'colsample': 0.5996864477108195, 'dense1_size': 500, 'dense2_size': 200, 'dense3_size': 400, 'dense4_size': 200, 'dense5_size': 300, 'learning_rate': 0.018962385872005834}
Epoch 10:                                             
	triggers: 0                                          
	best score: 0.9300898203592813                       
	6.3 seconds per epoch on average.                    
Epoch 20:                                             
	triggers: 2                                          
	best score: 0.9416823250051621                       
	6.33 seconds per epoch on average.                   
Epoch 30:                                             
	triggers: 4                                          
	best score: 0.948456535205451                        
	6.25 seconds per epoch on average.                   
Early stopping.                                       
{'GRU_size': 200, 'batch_size': 1500, 'colsample': 0.5996864477108195, 'den

In [22]:
trials.best_trial

{'state': 2,
 'tid': 69,
 'spec': None,
 'result': {'loss': -0.9563555647326037, 'status': 'ok'},
 'misc': {'tid': 69,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'GRU_size': [69],
   'dense1_size': [69],
   'dense2_size': [69],
   'dense3_size': [69],
   'dense4_size': [69],
   'dense5_size': [69]},
  'vals': {'GRU_size': [300.0],
   'dense1_size': [100.0],
   'dense2_size': [400.0],
   'dense3_size': [200.0],
   'dense4_size': [200.0],
   'dense5_size': [100.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2022, 6, 30, 12, 55, 6, 871000),
 'refresh_time': datetime.datetime(2022, 6, 30, 12, 57, 48, 623000)}

In [23]:
with open('second_search.pickle', 'wb') as file:
    pkl.dump(trials, file)